In [ ]:
import openmc
import numpy as np

### Constants

In [ ]:
BISO_KERNEL_RADIUS   = 0.04 # cm  # r = 400 μm = 0.0400 cm // "800 μm kernel"
BISO_RADIUS          = 0.05 # cm  # r = 500 μm = 0.0500 cm // "100 μm thickness"
BISO_VOLUME          = (4 / 3) * np.pi * (BISO_RADIUS)**3            # volume of single BISO particle
KERNEL_VOLUME        = (4 / 3) * np.pi * (BISO_KERNEL_RADIUS)**3     # volume of UO2/ThO2 kernel in single BISO particle
BISO_KERNEL_VOL_FRAC = KERNEL_VOLUME / BISO_VOLUME  # vol frac kernel in single BISO
BISO_COAT_VOL_FRAC   = 1.0 - BISO_KERNEL_VOL_FRAC

### Materials

In [ ]:
""" BISO particle """
# UO2 fuel kernel
uo2 = openmc.Material(name='UO2', temperature=900)
uo2.add_element('U', 1.0, enrichment=0.71)  
uo2.add_element('O', 2.0)
uo2.set_density('g/cm3', 10.5)

# SiC coating
sic = openmc.Material(name='SiC', temperature=900)
sic.add_element('Si', 1.0)
sic.add_element('C', 1.0)
sic.set_density('g/cm3', 3.2)

# BISO
biso = openmc.Material.mix_materials([uo2, sic], [BISO_KERNEL_VOL_FRAC, BISO_COAT_VOL_FRAC], 'vo')


""" Breeding material """
# Lead-lithium (Pb83at%-Li17at%)
pbli = openmc.Material(name='PbLi', temperature=900)
pbli.add_element('Pb', 0.83)
pbli.add_element('Li', 0.1, enrichment_target='Li6', enrichment_type='wo', enrichment=90.0)
pbli.set_density('g/cm3', 9.5) 

# FLiBe (LiF-BeF2)
flibe = openmc.Material(name='FLiBe', temperature=900)
flibe.add_elements_from_formula('F4Li2Be', 'ao', enrichment_target='Li6', enrichment_type='wo', enrichment=7.5)
flibe.set_density('g/cm3', 1.94) 


""" Homogeneous breeding mixtures """
flibe_mix_01  = openmc.Material.mix_materials([flibe, biso], [0.99, 0.01], 'vo')
flibe_mix_50  = openmc.Material.mix_materials([flibe, biso], [0.50, 0.50], 'vo')
pbli_mix_01  = openmc.Material.mix_materials([pbli, biso], [0.99, 0.01], 'vo')
pbli_mix_50  = openmc.Material.mix_materials([pbli, biso], [0.50, 0.50], 'vo')


""" Materials """
materials = openmc.Materials([uo2, sic, flibe, pbli, biso, mix])
materials.cross_sections = '/mnt/c/OpenMC/data/endfb-viii.0-hdf5/cross_sections.xml'
# materials.export_to_xml()

In [ ]:
def cube_dims(biso_vol_frac):
    cube_length = ((4/3 * np.pi * BISO_RADIUS**3)/biso_vol_frac)**(1/3)
    cube_half   = cube_length / 2
    return cube_length, cube_half

def periodic_box(xmin, xmax, ymin, ymax, zmin, zmax):
    xs = openmc.XPlane(x0=xmin, boundary_type = 'periodic')
    xg = openmc.XPlane(x0=xmax, boundary_type = 'periodic')

    ys = openmc.YPlane(y0=ymin, boundary_type = 'periodic')
    yg = openmc.YPlane(y0=ymax, boundary_type = 'periodic')

    zs = openmc.ZPlane(z0=zmin, boundary_type = 'periodic')
    zg = openmc.ZPlane(z0=zmax, boundary_type = 'periodic')

    xs.periodic_surface, ys.periodic_surface, zs.periodic_surface = xg, yg, zg

    region = +xs & -xg & +ys & -yg & +zs & -zg
    return region

In [ ]:
""" Cells """
# BISO particle
kernel_outer = openmc.Sphere(r=BISO_KERNEL_RADIUS) # UO2 kernel sphere (radius = 0.04 cm)
sic_outer = openmc.Sphere(r=BISO_RADIUS)           # # SiC coating outer surface (radius = 0.04 + 0.01 = 0.05 cm)

# Cubes
cube_length_01, cube_half_01 = cube_dims(0.01)
cube_length_50, cube_half_50 = cube_dims(0.50)
buffer = 1 # cm

box_centers = 

# FLiBe -  1 vol% - homogeneous 
lower_coords = (
upper_coords = 
cube_bounds = openmc.BoundingBox(

# FLiBe -  1 vol% - heterogeneous 

# FLiBe - 50 vol% - homogeneous 
lower_coords = (
upper_coords = 
cube_bounds = openmc.BoundingBox(

# FLiBe - 50 vol% - heterogeneous 

# LL -  1 vol% - homogeneous 
lower_coords = (
upper_coords = 
cube_bounds = openmc.BoundingBox(

# LL -  1 vol% - heterogeneous 

# LL - 50 vol% - homogeneous 
lower_coords = (
upper_coords = 
cube_bounds = openmc.BoundingBox(

# LL - 50 vol% - heterogeneous 
        

# Cubes
cells, pitch, half_box = [], 10, 50 # +/- 50 cm bounds
box_centers = [pitch * i for i in range(len(self.u_list))] # used in Sources

for i, material in enumerate(mix_list):
    x_min = openmc.XPlane(x0= -half_box + box_centers[i], boundary_type='periodic')
    x_max = openmc.XPlane(x0=  half_box + box_centers[i], boundary_type='periodic')
    y_min, y_max = openmc.YPlane(-50, boundary_type='periodic'), openmc.YPlane( 50, boundary_type='periodic')
    z_min, z_max = openmc.ZPlane(-50, boundary_type='periodic'), openmc.ZPlane( 50, boundary_type='periodic')
    region = +x_min & -x_max & +y_min & -y_max & +z_min & -z_max
    
    cell = openmc.Cell(fill=material, region=region)
    cell.name = f"mix-{i+1}"
    cells.append(cell)

root_univ = openmc.Universe(cells=cells) # Create root universe with all material cells
root_univ.plot(width=(pitch * len(self.u_list), 110), origin=(pitch * (len(self.u_list) - 1) / 2, 0.0, 0.0)) # Visualize
self.geometry = openmc.Geometry(root_univ) # Set geometry

In [ ]:
# Reflective cube boundaries 
cube_length = 1  # cm
cube_half   = cube_length / 2

x_min = openmc.XPlane(-cube_half, boundary_type='periodic')
x_max = openmc.XPlane( cube_half, boundary_type='periodic')
y_min = openmc.YPlane(-cube_half, boundary_type='periodic')
y_max = openmc.YPlane( cube_half, boundary_type='periodic')
z_min = openmc.ZPlane(-cube_half, boundary_type='periodic')
z_max = openmc.ZPlane( cube_half, boundary_type='periodic')
x_min.periodic_surface = x_max
y_min.periodic_surface = y_max
z_min.periodic_surface = z_max

# Homogeneous matrix
cell = openmc.Cell(name='mixture')
cell.fill = mix
cell.region = +x_min & -x_max & +y_min & -y_max & +z_min & -z_max

# Create universe and geometry
root_universe = openmc.Universe(cells=[cell])
geometry = openmc.Geometry(root_universe)
# geometry.export_to_xml()

In [ ]:
img = geometry.plot(basis='yz',                           # slice plane
                    width=(cube_length, cube_length),     # cm window
                    pixels=(200, 200),
                    color_by='material')

In [ ]:
settings = openmc.Settings()

"""Source: 14 MeV"""
source = openmc.IndependentSource()
source.space = openmc.stats.Box([-1*cube_half, -1*cube_half, -1*cube_half], [cube_half, cube_half, cube_half])
source.particle = 'neutron'
source.constraints = {'fissionable': False}
source.energy = openmc.stats.Discrete(energies, weights) # openmc.stats.Discrete([14.0e6], [1.0])  # 14 MeV
# source.angle    = openmc.stats.Monodirectional((1, 0, 0))
source.angle = openmc.stats.Isotropic()


"""Settings"""
settings.run_mode  = "fixed source"
settings.batches   = 1
settings.particles = int(1e8)
settings.source    = [source]
settings.volume_normalized_flux_tallies = True
# settings.write_initial_source = True
# settings.sourcepoint = {'batches':[10],'write':True}

In [ ]:
"""Tallies"""
tallies = openmc.Tallies()

# total rxn rates
t_tot = openmc.Tally(name="total rxn rates")
t_tot.scores = ["absorption", "fission", "(n,gamma)", "(n,Xt)"]
tallies.append(t_tot)

# rxn rates by isotope
t_iso = openmc.Tally(name="rxn rates by isotope")
t_iso.scores = ["absorption", "fission", "(n,gamma)", "(n,Xt)"]
t_iso.nuclides = ['U238', 'U235']
tallies.append(t_iso)

In [ ]:
model = openmc.Model(geometry, materials, settings, tallies)
model.export_to_model_xml("./BISO_LL_homogeneous")
model.run(cwd="./BISO_LL_homogeneous") 